In [1]:
%pip install langchain_community
%pip install azure-ai-ml
%pip install -qU langchain-openai
%pip install json5
%pip install evaluate
%pip install rouge_score

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.llms.azureml_endpoint import (
    AzureMLOnlineEndpoint,
    AzureMLEndpointApiType,
    DollyContentFormatter,
    ContentFormatterBase
)
from typing import Dict
import urllib.request
import json
import os
import ssl
import time
from langchain_openai import AzureChatOpenAI
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLEndpointApiType,
    CustomOpenAIChatContentFormatter,
)
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint

import json5
import re
import time
import requests



#### 1st Agent: Fine-tuned LLM

In [ ]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True)

class CustomFormatter(ContentFormatterBase):
    content_type = "application/json"
    accepts = "application/json"

    def format_request_payload(self, prompt: str, *args, **kwargs) -> bytes:
        model_kwargs = kwargs.get('model_kwargs', {})
        input_str = json.dumps(
            {"input_data": [{"role": "user", "content": prompt}], "params": {"temperature": 0.1, "max_new_tokens": 1000, "do_sample": True}}
        )
        return str.encode(input_str)

    def format_response_payload(self, output: bytes, *args, **kwargs) -> Dict:
        response_json =  json.loads(output)

        try:
            # Step 1: Prepare JSON string
            response_content = response_json['result'].replace("'", "\"")

            # Step 2: Escape double quotes within the `content` fields
            response_content = re.sub(r'(?<=content": ")', lambda m: m.group(0).replace('"', '\\"'), response_content)
            response_content = response_content.replace('Here"s', 'Here\'s')
            
            # Step 3: Split the JSON string based on "} {" separator to handle multiple entries
            json_entries = response_content.split("} {")
            
            # Wrap entries in braces to create valid JSON for each entry
            json_entries = ["{" + entry + "}" if not entry.startswith("{") else entry for entry in json_entries]
            json_entries = [entry + "}" if not entry.endswith("}") else entry for entry in json_entries]
            
            # Step 4: Parse each JSON entry and retrieve assistant content
            assistant_content = None
            for json_string in json_entries:
                # Regular expression to find "role": "assistant" and capture "content" value
                match = re.search(r'"role"\s*:\s*"assistant",\s*"content"\s*:\s*"(.*?)"\s*}', json_string, re.DOTALL)

                # Check if role is "assistant" and retrieve content
                if match:
                    assistant_content = match.group(1)  # Get the content text
        
            return {"text": assistant_content}
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")

        # Return a default response if there was an error
            return {"text": "No valid response"}

# Example usage of the CustomFormatter
content_formatter = CustomFormatter()


solver_llm = AzureMLOnlineEndpoint(
    endpoint_url="",
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key="",
    model_kwargs={"temperature": 0.1, "max_new_tokens": 1000, "do_sample": True},
    content_formatter=content_formatter,
)




#### Second Agent: RAG + Phi-3

In [ ]:
# Azure AI Search setup
AZURE_AI_SEARCH_SERVICE_NAME = ""
AZURE_AI_SEARCH_API_KEY = ""

retriever = AzureAISearchRetriever(
    service_name=AZURE_AI_SEARCH_SERVICE_NAME,
    index_name="vector-rag",
    api_key=AZURE_AI_SEARCH_API_KEY,
    content_key="chunk",
    top_k=1
)

# Azure OpenAI setup
coder_llm = AzureMLChatOnlineEndpoint(
    endpoint_url="",
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key="",
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"temperature": 0.1, "max_new_tokens": 1000},
)

ValidationError: 1 validation error for AzureAISearchRetriever
max_query_length
  Extra inputs are not permitted [type=extra_forbidden, input_value=2000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden

#### Reasoning Agent: Phi-3

In [ ]:
reasoning_agent = AzureMLChatOnlineEndpoint(
    endpoint_url="",
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key="",
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"temperature": 0.1, "max_new_tokens": 1000},
)

### Format csv to json

In [4]:
import pandas as pd
df = pd.read_csv("analysis_and_tools_only.csv")

In [7]:
#preprocess data

input_string = []

# Iterate through the rows of the dataframe
for index, row in df.iterrows():
    # 'content' is the column with the user questions
    user_content = row['content']
    # 'answer content' is the column with the ground truth answers
    assistant_content = row['answer_content']

    # Append only user content to the input_data
    input_string.append({"role": "user", "content": user_content})

# Define the parameters for the model
params = {
    "temperature": 0.1,
    "max_new_tokens": 4096, #output
    "do_sample": True,
    "return_full_text": False
}

# Combine input data and parameters into the final test data structure
test_data = {
    "input_data": input_string,
    "parameters": params
}

# Save the JSON output to a file
with open('test_data.json', 'w') as json_file:
    json.dump(test_data, json_file, indent=4)

print("JSON file created successfully.")

JSON file created successfully.


#### Muti-Agent Logic

In [73]:
def format_docs(docs):
    """Format retrieved documents into a single string."""
    return "\n\n".join(doc.page_content for doc in docs if doc.page_content) if docs else ""


In [140]:
def invoke_with_retries(llm, prompt, max_retries=5, backoff_factor=2):
    """
    Helper function to invoke LLM with retry logic in case of timeout or HTTP errors.
    """
    for attempt in range(max_retries):
        try:
            return llm.invoke(prompt, timeout=300000)  # Attempt to invoke LLM
        except TimeoutError:
            # Handle timeout errors with exponential backoff
            sleep_time = backoff_factor * (2 ** attempt)
            time.sleep(sleep_time)
            print(f"Timeout occurred. Retrying after {sleep_time} seconds... Attempt {attempt + 1}/{max_retries}")
        except HTTPError as e:
            if e.code == 424:  # Handle HTTP 424: Failed Dependency
                print(f"HTTP Error 424: Failed Dependency. Skipping after {attempt + 1}/{max_retries} attempts.")
                return None
            else:
                print(f"HTTP Error {e.code}: {e.reason}. Retrying...")
                sleep_time = backoff_factor * (2 ** attempt)
                time.sleep(sleep_time)
    print(f"Max retries reached for {llm}.")
    return None  # Return None if all retries fail

In [263]:
def chunk_query(query, chunk_size=1500):
    return [query[i:i+chunk_size] for i in range(0, len(query), chunk_size)]

In [268]:
def generate_final_response(query, tries=3, backoff_factor=1):
    attempt = 0
    final_response = ""
    while attempt < tries:

        # Step 1: Retrieve documents
        """Trim the string after the first occurrence of 'ETA:'."""
        eta_index = query.find("ETA:")
        if eta_index != -1:
            query = query[:eta_index].strip()
        query_chunks = chunk_query(query)
        docs = []
        for chunk in query_chunks:
            docs.extend(retriever.invoke(chunk))
        if not docs:
            print("No documents found for the query.")

        # Step 2: Format documents
        context = format_docs(docs)
        if not context:
            print("Formatted context is empty.")

        # Step 3: Generate prompt for coder LLM
        prompt = f"""This is relevant code documentation: \n{context}\n\nQuestion: {query}.\nPlease respond with code."""
        response_coder = coder_llm.invoke(prompt)

        # Process coder response
        response_code = response_coder.content if hasattr(response_coder, 'content') else ""
        if not response_code:
            print("Unexpected response from coder LLM.")

        # Step 4: Generate solver response with retry on timeout
        
        solver_text = None
        max_retries = 10
        retries = 0
        base_sleep = 5  # Start with a 5 second wait
        sleep_time = base_sleep
        
        while retries < max_retries and solver_text is None:
            solver_text = invoke_with_retries(solver_llm, query, max_retries=1)  # Retry logic for a single attempt
            if solver_text is None:
                print(f"Retry {retries + 1}/{max_retries} failed. Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
                retries += 1
                sleep_time *= 2  # Exponentially increase the sleep time for each retry

        if solver_text is None:
            print("Failed to get response from solver LLM after maximum retries.")
            solver_text = ""  # Fallback if all attempts fail

        # Step 5: Generate reasoning prompt
        prompt_reason = (
            f"You are an AI reasoning agent collaborating with other specialized assistants. "
            f"There are two previous responses you should review: "
            f"A solving agent specializing in bioinformatics tools has provided the following response: {solver_text}. "
            f"A coder agent specializing in genomics analysis workflows written in Nextflow has provided the following response: {response_code}. "
            f"Your task is to answer the user query: {query}. "
            f"After you generate your response, print the quality rating of your output as Quality Rating: on a new line. using the scale 1-10 using format x/10."
        )

        combined_reasoning = reasoning_agent.invoke(prompt_reason)

        # Step 6: Check reasoning response
        if hasattr(combined_reasoning, 'content'):
            final_response = combined_reasoning.content

            match = re.search(r'Quality Rating: (\d+)/10', final_response)

            if match:
                quality_rating = int(match.group(1))
                if quality_rating > 7:
                    print("Achieved satisfactory rating.")
                    return final_response, attempt # Return response if rating is above 7
                else:
                    print("Rating not satisfactory, continuing to next attempt.")
            else:
                print("Quality Rating not found in the response.")
        else:
            print("Reasoning agent did not return valid content.")

        # Backoff before the next attempt
        sleep_time = backoff_factor * (2 ** attempt)  # Exponential backoff
        print(f"Sleeping for {sleep_time} seconds before next attempt.")
        time.sleep(sleep_time)
        attempt += 1

    print("Max attempts reached. Returning last response.")
    return final_response, attempt


In [76]:
# Define the path for saving the real request file
test_src_dir = "./inference-test"
os.makedirs(test_src_dir, exist_ok=True)
real_data_path = os.path.join(test_src_dir, "test_data.json")

# Save the real request data to a JSON file
with open(real_data_path, "w") as f:
    json.dump(test_data, f, indent=4)

In [77]:
# Define paths and parameters
timeout = 1000
test_src_dir = "./inference-test"
response_src_dir = "./inference-responses"
batch_size = 1

# Ensure directories exist
os.makedirs(test_src_dir, exist_ok=True)
os.makedirs(response_src_dir, exist_ok=True)

def batch_data(data, batch_size):
    """Split data into batches of given size."""
    return [data[i:i + batch_size] for i in range(0, len(data), batch_size)]


# Create batches
batches = batch_data(test_data["input_data"], batch_size)

In [214]:
def submit_tasks(batch, id):

    combined_query = batch[0]['content']

    query = f"""Question: {combined_query}\nPlease provide a solution in raw text."""
    final_response, tries = generate_final_response(query)
    
     # Save both input and response data
    response_file_path = os.path.join(response_src_dir, f"response_{id}.json")
    with open(response_file_path, "w") as f:
        json.dump({
            "input_data": query,
            "response_data": final_response,
            "attempts": tries # Ensure result is JSON serializable
        }, f, indent=4)



In [275]:
import os
from urllib.error import HTTPError  # Adjust the import as needed based on your HTTP library

# Example batch processing
for i, batch in enumerate(batches):
    response_file_path = os.path.join(response_src_dir, f"response_{i}.json")
    
    if os.path.exists(response_file_path):
        print(f"The file '{response_file_path}' exists.")
    else:
        print(f"The file '{response_file_path}' does not exist.")
        print(f"Working on {i + 1} question")
        submit_tasks(batch, i)




The file './inference-responses/response_0.json' exists.
The file './inference-responses/response_1.json' exists.
The file './inference-responses/response_2.json' exists.
The file './inference-responses/response_3.json' exists.
The file './inference-responses/response_4.json' exists.
The file './inference-responses/response_5.json' exists.
The file './inference-responses/response_6.json' exists.
The file './inference-responses/response_7.json' exists.
The file './inference-responses/response_8.json' exists.
The file './inference-responses/response_9.json' exists.
The file './inference-responses/response_10.json' exists.
The file './inference-responses/response_11.json' exists.
The file './inference-responses/response_12.json' exists.
The file './inference-responses/response_13.json' exists.
The file './inference-responses/response_14.json' exists.
The file './inference-responses/response_15.json' exists.
The file './inference-responses/response_16.json' exists.
The file './inference-re

In [15]:
import glob
import pandas as pd

file_pattern = "./inference-responses/*.json"

questions = []
answers = []
attempts = []
quality_ratings = []


for file_name in glob.glob(file_pattern): #glob lists all the text files in the current working dir.
    with open(file_name, 'r') as file:
        data = json.load(file)
    

    if "input_data" in data: #retrieves list of dictionaries
        question = data["input_data"]
        questions.append(question)

    if "response_data" in data:
        response = data["response_data"]
        answers.append(response)
        match = re.search(r'Quality Rating: (\d+)/10', response)
        quality_rating = int(match.group(1))
        quality_ratings.append(quality_rating)

    if "attempts" in data:
        attempt = data["attempts"] + 1
        attempts.append(attempt)

final_df = pd.DataFrame({"Question": questions, "Answer": answers, "Attempts": attempts, 'Quality Rating': quality_ratings})

# Save DataFrame to a CSV file (optional)
final_df.to_csv('combined_questions_answers.csv', index=False)

final_df.head()

,Question,Answer,Attempts,Quality Rating
0,Question: I used Hmmer search for finding doma...,To overcome the issue of HMMER identifying dom...,1,9
1,"Question: I am looking for a good workflows, r...",To help you with SNP calling and understanding...,1,9
2,"Question: Dear all,I have indexed the C. elega...","Quality Rating: 8/10\n\nDear C,\n\nThank you f...",1,8
3,"Question: Hie NGS Geeks,Recently I observed so...",It seems like you're observing differences in ...,2,8
4,Question: How can I get the number of mapped r...,To get the number of mapped reads for a partic...,1,10


#### ROUGE

########note: rouge captures (number of n-grams in prediction summary (fine-tuned phi-3 model) that match the reference summary (ground-truth)) / number of n-grams in reference summary

In [6]:
import evaluate
predictions = final_df["Answer"].to_list()
references = df["answer_content"].to_list()

rouge = evaluate.load('rouge') #https://huggingface.co/spaces/evaluate-metric/rouge


results = rouge.compute(predictions=predictions,
                         references=references,
                        use_aggregator=True)

print("ROUGE-1:", round(results["rouge1"], 3))
print("ROUGE-2:", round(results["rouge2"], 3))
print("ROUGE-L:", round(results["rougeL"], 3))
print("ROUGE-Lsum:", round(results["rougeLsum"], 3))

/Users/venkatmalladi/opt/anaconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ROUGE-1: 0.121
ROUGE-2: 0.012
ROUGE-L: 0.071
ROUGE-Lsum: 0.086


In [11]:
rouge = pd.DataFrame.from_dict(results.items())

In [13]:
# Save DataFrame to a CSV file (optional)
rouge.to_csv('biostars_multiagent_rouge.csv', index=False)

In [255]:
query_part1= ["Student ask: 'How would I provide quality metric on fastq files?' Provide the steps you would advise the student to take.",
              "Student ask: 'What code or workflow do I need to write to provide quality metric on fastq files?' Provide the steps you would advise the student to take.",
              "Student ask: 'How do I align RNA-seq data against human reference genome?' Provide the steps you would advise the student to take.",
              "Student ask: 'What code or workflow do I need to write to align RNA-seq data against human reference genome?' Provide the steps you would advise the student to take.",
              "Student ask: 'How can I assemble, annotate, and analyze SARS-CoV-2 genomes from sequencing data to identify and characterize different variants of the virus?' Provide the steps you would advise the student to take.",
              "Student ask: 'What code or workflow do I need to write to assemble, annotate, and analyze SARS-CoV-2 genomes from sequencing data to identify and characterize different variants of the virus?' Provide the steps you would advise the student to take."
              ]

query_part2= "Please explain your logic and reasoning behind your answer."
query_part3 = "What additional information do you need to answer the question?"

results = {"easy_q" : "", "easy_code" : "", "med_q": "" , "med_code": "", "hard_q": "", "hard_code": ""}

for idx, question in enumerate (query_part1):

  combined_query = f"{question}\n{query_part2}\n{query_part3}"
  query = f"Question: {combined_query}\nPlease provide a solution in raw text."
  print (query)
  final_response = generate_final_response(query)

  if idx == 0:
    results["easy_q"] = final_response

  elif idx == 1:
    results["easy_code"] = final_response

  elif idx == 2:
    results["med_q"] = final_response

  elif idx == 3:
    results["med_code"] = final_response

  elif idx == 4:
    results["hard_q"] = final_response
 
  elif idx == 5:
    results["hard_code"] = final_response



Question: Student ask: 'How would I provide quality metric on fastq files?' Provide the steps you would advise the student to take.
Please explain your logic and reasoning behind your answer.
What additional information do you need to answer the question?
Please provide a solution in raw text.
Achieved satisfactory rating.
Question: Student ask: 'What code or workflow do I need to write to provide quality metric on fastq files?' Provide the steps you would advise the student to take.
Please explain your logic and reasoning behind your answer.
What additional information do you need to answer the question?
Please provide a solution in raw text.
Timeout occurred. Retrying after 2 seconds... Attempt 1/1
Max retries reached for AzureMLOnlineEndpoint
Params: {'deployment_name': '', 'model_kwargs': {'temperature': 0.1, 'max_new_tokens': 1000, 'do_sample': True, 'timeout': 300000}}.
Retry 1/10 failed. Retrying in 5 seconds...
Timeout occurred. Retrying after 2 seconds... Attempt 1/1
Max retri

In [17]:
for i in results:
    response_file_path =  f"response_{i}.json"
    with open(response_file_path, "w") as f:
        json.dump(results[i], f, indent=4)


0.12116104379602413
0.011580071165548536
0.07095299989507012
0.08641731222245205


In [22]:
file_pattern = "./response*.json"


for file_name in glob.glob(file_pattern): #glob lists all the text files in the current working dir.
    with open(file_name, 'r') as file:
        data = json.load(file)
        print(file_name)
        print(data[0])

./response_hard_code.json
To assemble, annotate, and analyze SARS-CoV-2 genomes from sequencing data to identify and characterize different variants of the virus, I would advise the student to follow these steps:

1. Obtain sequencing data: Obtain raw sequencing data from a reliable source, such as the NCBI or other public databases. Ensure that the data is of high quality and has been generated using a reliable sequencing platform.

2. Quality control: Perform quality control on the raw sequencing data using tools like FastQC to assess the quality of the reads and remove any low-quality reads or adapter sequences.

3. Read trimming: Trim the reads using tools like Trimmomatic or Cutadapt to remove low-quality bases and adapter sequences.

4. De novo assembly: Assemble the cleaned reads using a de novo assembly tool like SPAdes or MEGAHIT to generate contigs.

5. Genome annotation: Annotate the assembled contigs using tools like Prokka or RAST to identify genes and other functional ele